In [2]:
%pip install playwright pandas
!playwright install chromium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
GMAPS_URL = "https://www.google.com/maps/place/Grand+Mercure+Yogyakarta+Adi+Sucipto/@-7.7836348,110.3898571,17z/data=!4m9!3m8!1s0x2e7a59c53e8f9e4f:0x428ec2bf72c50217!5m2!4m1!1i2!8m2!3d-7.7836348!4d110.392432!16s%2Fg%2F11c5s5nkm4?entry=ttu&hl=id"

TARGET_REVIEWS = 5000       # belum bisa dapet sesuai target (kalo banyak)
CSV_PATH = "mercure.csv"
HEADLESS = False

# 🔽 SORT
# 0 = Paling relevan
# 1 = Terbaru
# 2 = Rating tertinggi
# 3 = Rating terendah
SORT_MODE = 2

In [12]:
import re
from playwright.async_api import async_playwright

async def crawl_reviews(url, target, existing_ids, sort_mode=1):
    new_data = []

    print("🚀 Launch browser")
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=HEADLESS,
            args=[
                "--disable-blink-features=AutomationControlled",
                "--no-sandbox",
                "--disable-dev-shm-usage",
            ]
        )

        page = await browser.new_page()

        print("🌐 Open Google Maps URL")
        await page.goto(url, timeout=60000)
        await page.wait_for_timeout(1000)
        await page.wait_for_selector("canvas", timeout=5000)
        await page.reload()
        await page.wait_for_timeout(1000)
        print("✅ Halaman kebuka")

        # Klik Reviews
        review_btn = page.locator(
            'button:has-text("Ulasan"), button:has-text("Reviews")'
        )

        if await review_btn.count() == 0:
            print("❌ Tombol Reviews tidak ketemu")
            await browser.close()
            return new_data

        await review_btn.first.click()
        await page.wait_for_timeout(2000)
        print("✅ Masuk tab Reviews")

        try:
            await page.wait_for_selector('div[data-review-id]', timeout=10000)
        except:
            print("⚠️ Review belum muncul, lanjut deteksi panel")

        # ======================
        # SORT
        # ======================
    
        
        print("🔀 Set sort ke TERBARU")

        try:
            # Tombol "Paling relevan"
            sort_btn = page.locator(
                'button[aria-label*="relevan"], button[aria-label*="relevance"], button:has-text("Paling relevan")'
            ).first

            await sort_btn.wait_for(state="visible", timeout=10000)
            await sort_btn.click()
            await page.wait_for_timeout(800)

            # pilih opsi berdasarkan data-index
            option = page.locator(
                f'div[role="menuitemradio"][data-index="{sort_mode}"]'
            ).first

            await option.wait_for(state="visible", timeout=10000)
            await option.click()
            await page.wait_for_timeout(2500)

            print("✅ Sort berhasil")

        except Exception as e:
            print(f"⚠️ Gagal set sort: {e}")

        async def detect_scrollable():
            selectors = [
                'div.m6QErb.DxyBCb.kA9KIf.dS8AEf',
                'div.m6QErb.XiKgde',
                'div[aria-label*="Reviews"]',
                'div[aria-label*="Ulasan"]',
            ]

            for s in selectors:
                el = page.locator(s).first
                if await el.count() and await el.locator('div[data-review-id]').count():
                    return el, s

            fallback = page.locator(
                'xpath=//div[@data-review-id]/ancestor::div[contains(@class,"scrollable")][1]'
            )
            if await fallback.count():
                return fallback.first, "xpath fallback"

            return None, None

        scrollable, selector = await detect_scrollable()

        if not scrollable:
            print("❌ Panel review tidak ketemu")
            await browser.close()
            return new_data

        print(f"✅ Scrollable ketemu pakai: {selector}")

        # ======================
        # SCROLL (ANTI NGACO)
        # ======================
        print("🔄 Scroll mulai")

        MAX_DOM = target + 20  # buffer
        last_count = 0
        stuck = 0

        while True:
            reviews = scrollable.locator('div[data-review-id]')
            count = await reviews.count()

            print(f"📦 Review di DOM: {count}")

            if count >= MAX_DOM:
                print("🛑 DOM cukup, stop scroll")
                break

            if count == last_count:
                stuck += 1
            else:
                stuck = 0

            if stuck >= 4:
                print("🛑 Scroll mentok")
                break

            last_count = count

            await scrollable.evaluate("el => el.scrollTop = el.scrollHeight")
            await page.wait_for_timeout(1200)

        # ======================
        # EXPAND REVIEW
        # ======================
        print("📖 Expand review panjang (smart mode)")

        expanded = 0
        stuck = 0

        while True:
            if len(new_data) >= target:
                print("   - Target sudah tercapai, stop expand")
                break

            more_btn = scrollable.locator(
                'button:has-text("Lainnya"), button:has-text("More")'
            ).first

            if await more_btn.count() == 0:
                print("   - Tidak ada tombol expand lagi")
                break

            # ⛔ cek apakah tombol ini terkait Tripadvisor
            btn_text = (await more_btn.inner_text()).lower()

            if "tripadvisor" in btn_text:
                print("   ⏭️ Skip expand (Tripadvisor button)")
                break
            try:
                await more_btn.click(timeout=1000)
                expanded += 1
                stuck = 0
                print(f"   - Expanded {expanded}")
                await page.wait_for_timeout(120)
            except:
                stuck += 1
                if stuck >= 3:
                    print("   - Expand mentok, stop")
                    break


        # ======================
        # EXTRACT DATA
        # ======================
        print("🧠 Extract data")

        reviews = scrollable.locator('div[data-review-id]')
        total = await reviews.count()
        print(f"📊 Total review di DOM: {total}")

        for i in range(total):
            print(f"\n➡️ Proses index {i}")

            if len(new_data) >= target:
                print("🛑 Target tercapai, break extract")
                break

            r = reviews.nth(i)

            # review_id
            try:
                review_id = await r.get_attribute("data-review-id")
                # print(f"   review_id: {review_id}")
            except Exception as e:
                print(f"   ❌ Gagal ambil review_id: {e}")
                continue

            if not review_id:
                print("   ⚠️ review_id kosong, skip")
                continue

            if review_id in existing_ids:
                print("   ⏭️ review_id sudah ada, skip")
                continue

            # rating
            rating = None
            try:
                rating_text = await r.locator(
                    'span.fontBodyLarge'
                ).first.text_content(timeout=800)

                if rating_text:
                    rating = int(rating_text.strip()[0])
                    print(f"   rating: {rating}")
                else:
                    print("   ⚠️ rating text kosong")

            except Exception as e:
                rating = None
                print(f"   ⚠️ rating gagal: {e}")


            # text
            text = ""

            try:
                container = r.locator('div.MyEned').first
                text = await container.text_content(timeout=1000)
                print(f"   text(MyEned): {text[:60]}...")
            except Exception as e:
                print(f"   ❌ gagal ambil text: {e}")
                continue

            text = (text or "").strip()

            if not text:
                print("   ⚠️ text kosong, skip")
                continue


            new_data.append({
                "review_id": review_id,
                "rating": rating,
                "review": text
            })
            existing_ids.add(review_id)

            print(f"   ✅ SAVED | total baru: {len(new_data)}")

                

        print(f"✅ Selesai. Review baru: {len(new_data)}")
        await browser.close()
        

    return new_data


## dibawah ini 3x crawling dengan sort berbeda biar dapet banyak

### get reviews (pakai sort "paling relevan")

In [ ]:
import os
import pandas as pd

In [ ]:
if os.path.exists(CSV_PATH):
    df_existing = pd.read_csv(CSV_PATH)
    existing_ids = set(df_existing["review_id"].astype(str))
    print(f"Resume mode ON — data lama: {len(existing_ids)} review")
else:
    # bikin CSV kosong
    df_existing = pd.DataFrame(columns=["review_id", "rating", "review"])
    df_existing.to_csv(CSV_PATH, index=False)

    existing_ids = set()
    print("CSV belum ada — file baru dibuat, mulai dari nol")


CSV belum ada — file baru dibuat, mulai dari nol


In [14]:
new_reviews = await crawl_reviews(GMAPS_URL, TARGET_REVIEWS, existing_ids, sort_mode=0)

df_new = pd.DataFrame(new_reviews)
df_final = pd.concat([df_existing, df_new], ignore_index=True)

df_final.to_csv(CSV_PATH, index=False)

print(f"Review baru ditambahkan: {len(df_new)}")
print(f"Total review sekarang: {len(df_final)}")


🚀 Launch browser
🌐 Open Google Maps URL
✅ Halaman kebuka
✅ Masuk tab Reviews
🔀 Set sort ke TERBARU
✅ Sort berhasil
✅ Scrollable ketemu pakai: div.m6QErb.DxyBCb.kA9KIf.dS8AEf
🔄 Scroll mulai
📦 Review di DOM: 20
📦 Review di DOM: 40
📦 Review di DOM: 60
📦 Review di DOM: 80
📦 Review di DOM: 100
📦 Review di DOM: 120
📦 Review di DOM: 140
📦 Review di DOM: 160
📦 Review di DOM: 180
📦 Review di DOM: 200
📦 Review di DOM: 220
📦 Review di DOM: 240
📦 Review di DOM: 260
📦 Review di DOM: 280
📦 Review di DOM: 300
📦 Review di DOM: 320
📦 Review di DOM: 340
📦 Review di DOM: 360
📦 Review di DOM: 380
📦 Review di DOM: 400
📦 Review di DOM: 420
📦 Review di DOM: 440
📦 Review di DOM: 460
📦 Review di DOM: 480
📦 Review di DOM: 500
📦 Review di DOM: 520
📦 Review di DOM: 540
📦 Review di DOM: 560
📦 Review di DOM: 580
📦 Review di DOM: 600
📦 Review di DOM: 620
📦 Review di DOM: 640
📦 Review di DOM: 660
📦 Review di DOM: 680
📦 Review di DOM: 700
📦 Review di DOM: 720
📦 Review di DOM: 740
📦 Review di DOM: 760
📦 Review di DOM: 

### get reviews (pakai sort "terbaru")

In [ ]:
if os.path.exists(CSV_PATH):
    df_existing = pd.read_csv(CSV_PATH)
    existing_ids = set(df_existing["review_id"].astype(str))
    print(f"Resume mode ON — data lama: {len(existing_ids)} review")
else:
    # bikin CSV kosong
    df_existing = pd.DataFrame(columns=["review_id", "rating", "review"])
    df_existing.to_csv(CSV_PATH, index=False)

    existing_ids = set()
    print("CSV belum ada — file baru dibuat, mulai dari nol")


Resume mode ON — data lama: 1100 review


In [16]:
new_reviews = await crawl_reviews(GMAPS_URL, TARGET_REVIEWS, existing_ids, sort_mode=1)

df_new = pd.DataFrame(new_reviews)
df_final = pd.concat([df_existing, df_new], ignore_index=True)

df_final.to_csv(CSV_PATH, index=False)

print(f"Review baru ditambahkan: {len(df_new)}")
print(f"Total review sekarang: {len(df_final)}") 

🚀 Launch browser
🌐 Open Google Maps URL
✅ Halaman kebuka
✅ Masuk tab Reviews
🔀 Set sort ke TERBARU
✅ Sort berhasil
✅ Scrollable ketemu pakai: div.m6QErb.DxyBCb.kA9KIf.dS8AEf
🔄 Scroll mulai
📦 Review di DOM: 20
📦 Review di DOM: 40
📦 Review di DOM: 60
📦 Review di DOM: 80
📦 Review di DOM: 100
📦 Review di DOM: 120
📦 Review di DOM: 140
📦 Review di DOM: 160
📦 Review di DOM: 180
📦 Review di DOM: 200
📦 Review di DOM: 220
📦 Review di DOM: 240
📦 Review di DOM: 260
📦 Review di DOM: 280
📦 Review di DOM: 300
📦 Review di DOM: 320
📦 Review di DOM: 340
📦 Review di DOM: 360
📦 Review di DOM: 380
📦 Review di DOM: 400
📦 Review di DOM: 420
📦 Review di DOM: 440
📦 Review di DOM: 460
📦 Review di DOM: 480
📦 Review di DOM: 500
📦 Review di DOM: 520
📦 Review di DOM: 540
📦 Review di DOM: 560
📦 Review di DOM: 580
📦 Review di DOM: 600
📦 Review di DOM: 620
📦 Review di DOM: 640
📦 Review di DOM: 660
📦 Review di DOM: 680
📦 Review di DOM: 700
📦 Review di DOM: 720
📦 Review di DOM: 740
📦 Review di DOM: 760
📦 Review di DOM: 

### get reviews (pakai sort "rating tertinggi")

In [ ]:
if os.path.exists(CSV_PATH):
    df_existing = pd.read_csv(CSV_PATH)
    existing_ids = set(df_existing["review_id"].astype(str))
    print(f"Resume mode ON — data lama: {len(existing_ids)} review")
else:
    # bikin CSV kosong
    df_existing = pd.DataFrame(columns=["review_id", "rating", "review"])
    df_existing.to_csv(CSV_PATH, index=False)

    existing_ids = set()
    print("CSV belum ada — file baru dibuat, mulai dari nol")


Resume mode ON — data lama: 1312 review


In [18]:
new_reviews = await crawl_reviews(GMAPS_URL, TARGET_REVIEWS, existing_ids, sort_mode=2)

df_new = pd.DataFrame(new_reviews)
df_final = pd.concat([df_existing, df_new], ignore_index=True)

df_final.to_csv(CSV_PATH, index=False)

print(f"Review baru ditambahkan: {len(df_new)}")
print(f"Total review sekarang: {len(df_final)}")

🚀 Launch browser
🌐 Open Google Maps URL
✅ Halaman kebuka
✅ Masuk tab Reviews
🔀 Set sort ke TERBARU
✅ Sort berhasil
✅ Scrollable ketemu pakai: div.m6QErb.DxyBCb.kA9KIf.dS8AEf
🔄 Scroll mulai
📦 Review di DOM: 20
📦 Review di DOM: 40
📦 Review di DOM: 60
📦 Review di DOM: 80
📦 Review di DOM: 100
📦 Review di DOM: 120
📦 Review di DOM: 140
📦 Review di DOM: 160
📦 Review di DOM: 180
📦 Review di DOM: 200
📦 Review di DOM: 220
📦 Review di DOM: 240
📦 Review di DOM: 260
📦 Review di DOM: 280
📦 Review di DOM: 300
📦 Review di DOM: 320
📦 Review di DOM: 340
📦 Review di DOM: 360
📦 Review di DOM: 380
📦 Review di DOM: 400
📦 Review di DOM: 420
📦 Review di DOM: 440
📦 Review di DOM: 460
📦 Review di DOM: 480
📦 Review di DOM: 500
📦 Review di DOM: 520
📦 Review di DOM: 540
📦 Review di DOM: 560
📦 Review di DOM: 580
📦 Review di DOM: 600
📦 Review di DOM: 620
📦 Review di DOM: 640
📦 Review di DOM: 660
📦 Review di DOM: 680
📦 Review di DOM: 700
📦 Review di DOM: 720
📦 Review di DOM: 740
📦 Review di DOM: 760
📦 Review di DOM: 